# ATMS_api

> functions for access ATMS_api

In [ ]:
#| default_exp ATMS_api

In [ ]:
#| exporti
from nbdev.showdoc import *
import requests
import json
import jmespath as jp
import re
from time import sleep
from fastcore.basics import patch
import fileinput
import pandas as pd

In [ ]:
#| export

class ATMS_api:
    def __init__(self):
        self.telus_access_token = ATMS_api.get_atms_authentication()
        self.obj_d = {}
    
    @staticmethod    
    def get_atms_authentication():
        """get access token for ATMS API

        Returns:
            response object: response object from the API call 
        """
        vantix_url = "http://crm-api-telus.atmsplus.com/auth"
        
        with open('secrets.json') as f:
            secrets = json.load(f)

        payload = json.dumps({
            "username": secrets['vantix_user'],
            "password": secrets['vantix_pw'],
            "rememberMe": True
        })
        headers = {
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", vantix_url, headers=headers, data=payload)
        assert response.status_code == 200, f"response code is {response.status_code}, not 200"
        return response.json().get('access_token')

                
    #make a function that takes a string and returns a string with the original semi-colon separated emails replaced with a list of emails with quotes around each
    @staticmethod
    def __mutate_email_list(s0:str) -> str :
        # fix "O'brien" data 
        s = re.sub('\u2019',"'",s0)
        pat_s = r"""\"emails\": \[.*?address\": (\"(?P<email1>.*?)\")"""
        pattern=re.compile(pat_s)
        matches = re.search(pattern,s) 
        if matches and matches.group(0) and matches.group(1) and matches.group(2):
            og_emails_list_s = matches.group(2)
            emails_l = [f"\"{email}\"" for email in og_emails_list_s.split(';')]
            # emails_l = [f"{email}" for email in og_emails_list_s.split(';')]
            emails_list_s = '[' +','.join(emails_l)+']'
            return re.sub(matches.group(1), emails_list_s,s)
        else:
            return s

    def __get_telus_data(self, obj, offset=0, count=1000):
        """retrieve data from ATMS API, should be private method

        Args:
            obj (string): api endpoint to retrieve data from
            offset (int, optional): first row to begin retrieval. Defaults to 0.
            count (int, optional): number of rows to retrieve. Defaults to 1000.

        Returns:
            response object: response object from the API call
        """
        vantix_data_url = f"http://crm-api-telus.atmsplus.com/api/{obj}?offset={offset}&count={count}"
        v_headers = {'Authorization': f"Bearer {self.telus_access_token}"}

        print(vantix_data_url)    
        response = requests.request("GET", vantix_data_url, headers=v_headers, data={}).json()
        
        # inform caller we're done if we get fewer records than requested
        return {"response": response, "done":  len(response) < count}

    def write_obj_to_file(self, obj, initial_offset=0, max_offset=2000, count=1000):
        """Retrieve data from ATMS API and write to file
           public method

        Args:
            obj (string): a valid ATMS REST API object
            max_offset (int, optional): starting row to begin retrieval. Defaults to 2000.
            count (int, optional): maximum number of rows to retieve. Defaults to 1000.
        """
        done = False
        offset = initial_offset 
        filename_s = f'atms_{obj}.json'
        print("Writing to file: ", filename_s)
        with open(filename_s, 'w') as f:
            f.write("[ \n")
            print('max_offset: ', max_offset)
            print('offset: ', offset)
            while (not done and  offset < max_offset):
                print('offset: ', offset)
                # read another file
                resp_d = self.__get_telus_data(obj,offset=offset, count=count)
                obj_l = resp_d['response']
                done = resp_d['done'] #or offset > max_offset
                for i,r in enumerate(obj_l):
                    ## this is so ugly...
                    line_ending = "\n" if done or (offset >= max_offset-offset and i>=len(obj_l)-1) else ",\n"
                    f.write(json.dumps(r)+ line_ending)
                offset += count
            f.write("]")

        # write_atms_obj_to_file('items', max_offset=1000, count=1000)
    
    
    # def clean_obj_file(self, obj_s : str): 
    def clean_data_file(self, obj_s : str): 
        """clean up the atms_contacts.json file, this is necessary before loading into Saleforce
        
        """
        # read original contacts file   
        in_filename_s = f'atms_{obj_s}.json'
        print("Cleaning file: ", in_filename_s)
        with open(in_filename_s,'r') as f:
            # write modified contacts file 
            out_filename_s = f'atms_transformed_{obj_s}.json'
            print("Writing file: ", out_filename_s)
            with open(out_filename_s,'w') as f2:
                s = f.read()
                for l in s.split('\n'):
                    new_s = ATMS_api.__mutate_email_list(l)+'\n'
                    f2.write(new_s)
               

In [ ]:
def test_mutate_email() -> bool:
    """ Test to cure backtick string in O'Brien """
    s ="225805	119016	2	06416302	Active	498585	O’Brien, Adam"
    assert ATMS_api.__mutate_email_list(s) == "225805\t119016\t2\t06416302\tActive\t498585\tO'Brien, Adam"

In [ ]:
#| export 

# make dict from json file

@patch
def load_data_file_to_dict(
        self: ATMS_api, 
        obj_s : str
        ):
    file_name_s = f'atms_transformed_{obj_s}.json'
    print('Loading file: ', file_name_s)
    with open(file_name_s,'r') as f2:
        s2 = f2.read()
        self.obj_d[obj_s] = json.loads(s2)
            

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()